<a href="https://colab.research.google.com/github/Arpit1118/Pytorch/blob/main/RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import numpy as np

In [15]:
class SyntheticDataset(Dataset):
    def __init__(self, num_samples=10000, seq_length=20, vocab_size=50):
        self.num_samples = num_samples
        self.seq_length = seq_length
        self.vocab_size = vocab_size

        # Generate random sequences and labels
        self.sequences = np.random.randint(1, vocab_size, size=(num_samples, seq_length))
        self.labels = np.random.randint(0, 2, size=num_samples)  # Binary labels (0 or 1)

    def __len__(self):
        return self.num_samples

    def __getitem__(self, idx):
        return torch.tensor(self.sequences[idx], dtype=torch.long), torch.tensor(self.labels[idx], dtype=torch.long)

# Generate a synthetic dataset with 10,000 samples, each with a sequence length of 20
dataset = SyntheticDataset(num_samples=10000, seq_length=20, vocab_size=100)
dataloader = DataLoader(dataset, batch_size=64, shuffle=True)


In [16]:
class RNNModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_size, output_size, num_layers=1):
        super(RNNModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.rnn = nn.RNN(embedding_dim, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        # x shape: (batch_size, seq_length)
        embedded = self.embedding(x)  # Shape: (batch_size, seq_length, embedding_dim)
        rnn_out, _ = self.rnn(embedded)  # RNN output shape: (batch_size, seq_length, hidden_size)
        out = rnn_out[:, -1, :]  # Get the output of the last time step (batch_size, hidden_size)
        out = self.fc(out)  # Shape: (batch_size, output_size)
        return out


In [17]:
# Parameters
vocab_size = 100  # Vocabulary size (number of unique words)
embedding_dim = 32  # Dimension of the word embeddings
hidden_size = 64  # Hidden size of the RNN
output_size = 2  # Binary classification (0 or 1)
num_epochs = 10
learning_rate = 0.001

# Instantiate the model, loss function, and optimizer
model = RNNModel(vocab_size, embedding_dim, hidden_size, output_size)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    correct_predictions = 0
    total_samples = 0

    for seq, label in dataloader:
        # Forward pass
        optimizer.zero_grad()
        outputs = model(seq)

        # Compute loss and backpropagate
        loss = criterion(outputs, label)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

        # Calculate accuracy
        _, predicted = torch.max(outputs, 1)
        correct_predictions += (predicted == label).sum().item()
        total_samples += label.size(0)

    avg_loss = total_loss / len(dataloader)
    accuracy = correct_predictions / total_samples * 100
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}, Accuracy: {accuracy:.2f}%')


Epoch [1/10], Loss: 0.6991, Accuracy: 50.36%
Epoch [2/10], Loss: 0.6934, Accuracy: 52.33%
Epoch [3/10], Loss: 0.6913, Accuracy: 52.74%
Epoch [4/10], Loss: 0.6897, Accuracy: 53.46%
Epoch [5/10], Loss: 0.6877, Accuracy: 54.44%
Epoch [6/10], Loss: 0.6847, Accuracy: 55.38%
Epoch [7/10], Loss: 0.6829, Accuracy: 55.64%
Epoch [8/10], Loss: 0.6798, Accuracy: 57.06%
Epoch [9/10], Loss: 0.6752, Accuracy: 57.60%
Epoch [10/10], Loss: 0.6692, Accuracy: 59.02%


In [18]:
# Testing the model
model.eval()
test_sequence = torch.tensor([[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]], dtype=torch.long)
with torch.no_grad():
    output = model(test_sequence)
    predicted_label = torch.argmax(output, dim=1)
    print(f'Predicted label: {predicted_label.item()}')


Predicted label: 0
